In [1]:
!git clone https://github.com/saeedahmadian/npy_desired_regular

Cloning into 'npy_desired_regular'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [3]:
!ls npy_desired_regular

desired_data.npy  README.md


In [151]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as plt3d

In [152]:
from sklearn.model_selection import train_test_split
data =np.load('npy_desired_regular/desired_data.npy')

In [153]:
x = data[:,0:2].reshape((-1,2))
y = data[:,2].reshape((-1,1))

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.3)

In [155]:
N_feature=2
N_out=1

In [156]:
with tf.name_scope('input'):
    x_tf = tf.placeholder(shape=[None,N_feature],dtype= tf.float32, name='x_input')
    y_tf = tf.placeholder(shape=[None,N_out],dtype= tf.float32,name='y_input')

In [198]:
with tf.name_scope('weights'):
#     w= tf.Variable(tf.truncated_normal([N_feature,N_out]),dtype= tf.float32, name='W')
#     b= tf.Variable(tf.truncated_normal([N_out]),dtype= tf.float32, name='b')
    w= tf.Variable(tf.zeros([N_feature,N_out]),dtype= tf.float32, name='W')
    b= tf.Variable(tf.zeros([N_out]),dtype= tf.float32, name='b')

In [199]:
with tf.name_scope('prediction'):
    y_pred = tf.add(tf.matmul(x_tf,w),b)
        

In [200]:
loss = tf.reduce_mean(tf.square(y_pred-y_tf))

In [201]:
variable_list=[w,b]

In [202]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=.1)

# 1-you can directly use minimize with respect to your variables

In [168]:
train_opt= optimizer.minimize(loss,var_list=variable_list)
"or"

'or'

# 2- You can see gradients and variables and do what you want to do

In [184]:
gradients, variables = zip(*optimizer.compute_gradients(loss))
train_op = optimizer.apply_gradients(list(zip(gradients, variables)))


# 3- You can apply resterictions on your gradients based on whatever you need to do

In [213]:
# 
gvs = optimizer.compute_gradients(loss,variable_list)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs)

In [143]:
grad_w= tf.gradients(loss,w)

In [144]:
grad_b = tf.gradients(loss,b)

In [166]:
import random
def sampling(data,batch_size):
    return np.array(random.sample(list(data),batch_size))


In [146]:
saver = tf.train.Saver()
tf.summary.histogram("weights",w)
tf.summary.histogram("bias",w)
tf.summary.scalar("loss",loss)
tf.summary.scalar("grad_w",grad_w)
tf.summary.scalar("grad_b",grad_b)
all_summaries=tf.summary.merge_all()

In [216]:
epochs= 3
batch_size=2
with tf.Session() as sess:
#     summ_write=tf.summary.FileWriter('summary',sess.graph)
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        x_batch= sampling(x_train,batch_size)
        y_batch = sampling(y_train,batch_size)
#         print(x_batch.shape)
#         print(x_batch)
#         print(y_batch),dloss_dw,dloss_db, weights,bias
        model, grads_vars,loss_value,y_predicted,weights,bias= sess.run([train_opt,capped_gvs,loss,y_pred,w,b],feed_dict={x_tf:x_batch,y_tf:y_batch})
        print('epoch: {}, loss: {}'.format(epoch,loss_value))
#         print(model_info)
#         print('weights are :')
#         print(weights)
#         print('bias are : ')
#         print(bias)
#         print('grads with respect to w are:')
#         print(dloss_dw)
#         print('grads with respect to bias is:')
#         print(dloss_db)

epoch: 0, loss: 147996.5
epoch: 1, loss: 340932.5
epoch: 2, loss: 186605.0


In [217]:
grads_vars

[(array([[-1.],
         [-1.]], dtype=float32), array([[0.],
         [0.]], dtype=float32)),
 (array([-1.], dtype=float32), array([0.], dtype=float32))]

In [206]:
weights

array([[0.],
       [0.]], dtype=float32)

In [207]:
bias

array([0.], dtype=float32)

In [212]:
grads_vars[0][0].shape

(2, 1)

In [208]:
for grad,var in grads_vars:
    print('gradient is :')
    print(grad)
    print('variavle is :')
    print(var)
    

gradient is :
[[-1034886.]
 [ -817944.]]
variavle is :
[[0.]
 [0.]]
gradient is :
[-1038.]
variavle is :
[0.]
